# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [4]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [7]:
!type talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
"Gender, style-shifting and authenticity in speech: insights from trans speakers"	Talk	talk-1	"Language Variation and Change Research Group, The University of Edinburgh"	2025-10-13	Edinburgh		
Identity and ideology in commercial TTS	Talk	talk-2	"Division of Speech, Music and Hearing (TMH) Seminar at KTH Royal Institute of Technology"	2025-09-17	Stockholm		
Conveying gender through speech: insights from trans men	Conference proceedings talk	talk-3	"Lavender Languages and Linguistics 31, Manchester Metropolitan University"	2025-08-21	Manchester		
Conveying gender through speech: insights from trans men	Conference proceedings talk	talk-4	Interspeech Special Session on Queer and Trans Speech Science and Technology	2025-08-18	Rotterdam		With Cliodhna Hughes
Ideology and discrimination in commercial text-to-speech	Talk	talk-5	"Center for Humans and Machines lab, Max-Planck-Institut für Bildungsforschung"	2025-08-06	Berlin		
Conveyi

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,"Gender, style-shifting and authenticity in spe...",Talk,talk-1,"Language Variation and Change Research Group, ...",2025-10-13,Edinburgh,NaN,NaN
1,Identity and ideology in commercial TTS,Talk,talk-2,"Division of Speech, Music and Hearing (TMH) Se...",2025-09-17,Stockholm,NaN,NaN
2,Conveying gender through speech: insights from...,Conference proceedings talk,talk-3,"Lavender Languages and Linguistics 31, Manches...",2025-08-21,Manchester,NaN,NaN
3,Conveying gender through speech: insights from...,Conference proceedings talk,talk-4,Interspeech Special Session on Queer and Trans...,2025-08-18,Rotterdam,NaN,With Cliodhna Hughes
4,Ideology and discrimination in commercial text...,Talk,talk-5,"Center for Humans and Machines lab, Max-Planck...",2025-08-06,Berlin,NaN,NaN
5,Conveying gender through speech: insights from...,Conference proceedings talk,talk-6,"UK and Ireland Speech, University of York",2025-06-16,York,NaN,NaN
6,Speaking Out: Three projects related to speech...,Talk,talk-7,"Centre for Speech Technology Research Talks, U...",2025-05-26,Edinburgh,NaN,NaN
7,Beyond the binary: Limitations and possibiliti...,Talk,talk-8,"Gender and Social Cognition reading group, Uni...",2025-04-14,Edinburgh,NaN,With Ariadna Sanchez
8,"Ideology, power and bias in the design of 'hum...",Talk,talk-9,ISCA/ITG Workshop on Diversity in Large Speech...,2025-02-20,Berlin,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [11]:
!dir ..\_talks

 Volume in drive C is OS
 Volume Serial Number is 8A52-B2D5

 Directory of c:\Users\alice\Documents\GitHub\alice-ross.github.io\_talks

29/10/2025  12:49    <DIR>          .
29/10/2025  12:39    <DIR>          ..
29/10/2025  11:11               363 2012-03-01-talk-1.md
29/10/2025  11:11               460 2013-03-01-tutorial-1.md
29/10/2025  11:11               385 2014-02-01-talk-2.md
29/10/2025  11:11               420 2014-03-01-talk-3.md
29/10/2025  12:49               402 2025-02-20-talk-9.md
29/10/2025  12:49               326 2025-04-14-talk-8.md
29/10/2025  12:49               268 2025-05-26-talk-7.md
29/10/2025  12:49               258 2025-06-16-talk-6.md
29/10/2025  12:49               274 2025-08-06-talk-5.md
29/10/2025  12:49               322 2025-08-18-talk-4.md
29/10/2025  12:49               296 2025-08-21-talk-3.md
29/10/2025  12:49               270 2025-09-17-talk-2.md
29/10/2025  12:49               295 2025-10-13-talk-1.md
29/10/2025  12:28               295 2025-t

In [12]:
!type ..\_talks\2025-talk-1.md

---
title: "Gender, style-shifting and authenticity in speech: insights from trans speakers"
collection: talks
type: "Talk"
permalink: /talks/13/10/2025-talk-1
venue: "Language Variation and Change Research Group, The University of Edinburgh"
date: 13/10/2025
location: "Edinburgh"
---
